# Lab 6: Performance — Column Partitioning

This lab compares query performance between a standard (non-partitioned) table and a column-partitioned table inside a Direct Lake semantic model. You will run the same set of DAX query patterns against both tables, capturing server timing metrics each time, so you can measure the impact of column partitioning on cold-cache and warm-cache execution.

**Prerequisites:** Lab 2 must be completed (Big Data lakehouse and semantic model).

---

*Dieses Lab vergleicht die Abfrageleistung zwischen einer Standard-Tabelle (nicht partitioniert) und einer spaltenpartitionierten Tabelle in einem Direct Lake Semantic Model. Sie fuehren dieselben DAX-Abfragemuster gegen beide Tabellen aus, erfassen jeweils Server-Timing-Metriken und koennen so die Auswirkungen der Spaltenpartitionierung auf die Cold-Cache- und Warm-Cache-Ausfuehrung messen.*

*Voraussetzung: Lab 2 muss abgeschlossen sein (Big Data Lakehouse und Semantic Model).*

## Step 1: Install Semantic Link Labs

Install the Semantic Link Labs Python library used throughout this lab.

---

*Installieren Sie die Semantic Link Labs Python-Bibliothek, die in diesem Lab verwendet wird.*

In [ ]:
%pip install -q semantic-link-labs

## Step 2: Load Python Libraries

Import the required libraries and attach to the BigData lakehouse.

---

*Importieren Sie die erforderlichen Bibliotheken und verbinden Sie sich mit dem BigData-Lakehouse.*

In [ ]:
# Import libraries for performance analysis and column partitioning optimization
import sempy_labs as labs
from sempy import fabric
import sempy
import pandas
import time
import warnings

# Configure BigData lakehouse for column partitioning experiments
LakehouseName = "BigData"
lakehouses = labs.list_lakehouses()["Lakehouse Name"]
for l in lakehouses:
    if l.startswith("Big"):
        LakehouseName = l

# Set up semantic model for performance testing
SemanticModelName = f"{LakehouseName}_model"

## Step 3: Setup Parameters

Define workspace and lakehouse identifiers used by subsequent steps.

---

*Definieren Sie die Workspace- und Lakehouse-Bezeichner, die in den folgenden Schritten verwendet werden.*

In [ ]:
# Validate BigData lakehouse exists (required from Lab 2)
lakehouses=labs.list_lakehouses()["Lakehouse Name"]
if LakehouseName in lakehouses.values:
    lakehouseId = notebookutils.lakehouse.getWithProperties(LakehouseName)["id"]
else:
    print("You need to complete Lab 2 to create the required lakehouse for this lab")

# Configure workspace parameters for performance testing
workspaceId = notebookutils.lakehouse.getWithProperties(LakehouseName)["workspaceId"]
workspaceName = sempy.fabric.resolve_workspace_name(workspaceId)
print(f"WorkspaceId = {workspaceId}, LakehouseID = {lakehouseId}, Workspace Name = {workspaceName}")

## Step 4: Create DAX Query Function with Server Timings

Build a helper function that executes a DAX query while capturing server timing traces. This function is used in later steps to measure cold-cache and warm-cache performance.

---

*Erstellen Sie eine Hilfsfunktion, die eine DAX-Abfrage ausfuehrt und dabei Server-Timing-Traces erfasst. Diese Funktion wird in spaeteren Schritten zur Messung der Cold-Cache- und Warm-Cache-Leistung verwendet.*

In [ ]:
from Microsoft.AnalysisServices.Tabular import TraceEventArgs
from typing import Dict, List, Optional, Callable

#### Generate Unique Trace Name - Start ####
import json, base64, re
token = notebookutils.credentials.getToken("pbi")
payload = token.split(".")[1]
payload += "=" * (4 - len(payload) % 4)
upn = json.loads(base64.b64decode(payload)).get("upn")

# Extract just the user part (e.g. "SQLKDL.user39")
user_id = upn.split("@")[0]
lab_number = 6  # set per lab

# Remove characters not allowed in trace names: . , ; ' ` : / \ * | ? " & % $ ! + = ( ) [ ] { } < >
user_id_clean = re.sub(r"[.,;'`:/\\*|?\"&%$!+=(){}\[\]<>]", "_", user_id)
trace_name = f"Lab{lab_number}_{user_id_clean}"
#### Generate Unique Trace Name - End ####


def runDMV():
    df = sempy.fabric.evaluate_dax(
        dataset=SemanticModelName, 
        dax_string="""
        
        SELECT 
            MEASURE_GROUP_NAME AS [TABLE],
            ATTRIBUTE_NAME AS [COLUMN],
            DATATYPE ,
            DICTIONARY_SIZE 		    AS SIZE ,
            DICTIONARY_ISPAGEABLE 		AS PAGEABLE ,
            DICTIONARY_ISRESIDENT		AS RESIDENT ,
            DICTIONARY_TEMPERATURE		AS TEMPERATURE,
            DICTIONARY_LAST_ACCESSED	AS LASTACCESSED 
        FROM $SYSTEM.DISCOVER_STORAGE_TABLE_COLUMNS 
        ORDER BY 
            [DICTIONARY_TEMPERATURE] DESC
        
        """)
    display(df)

def filter_func(e):
    retVal:bool=True
    if e.EventSubclass.ToString() == "VertiPaqScanInternal":
        retVal=False      
    #     #if e.EventSubClass.ToString() == "VertiPaqScanInternal":
    #     retVal=False
    return retVal

# define events to trace and their corresponding columns
def runQueryWithTrace (expr:str,workspaceName:str,SemanticModelName:str,Result:Optional[bool]=True,Trace:Optional[bool]=True,DMV:Optional[bool]=True,ClearCache:Optional[bool]=True) -> pandas.DataFrame :
    event_schema = fabric.Trace.get_default_query_trace_schema()
    event_schema.update({"ExecutionMetrics":["EventClass","TextData"]})
    del event_schema['VertiPaqSEQueryBegin']
    del event_schema['VertiPaqSEQueryCacheMatch']
    del event_schema['DirectQueryBegin']

    warnings.filterwarnings("ignore")

    if ClearCache:
        labs.clear_cache(SemanticModelName)

    WorkspaceName:str = workspaceName
    SemanticModelName:str = SemanticModelName

    with fabric.create_trace_connection(SemanticModelName,WorkspaceName) as trace_connection:
        # create trace on server with specified events
        with trace_connection.create_trace(
            event_schema=event_schema, 
            name=trace_name,
            filter_predicate=filter_func,
            stop_event="QueryEnd"
            ) as trace:

            trace.start()

            df:FabricDataFrame=sempy.fabric.evaluate_dax(
                dataset=SemanticModelName, 
                dax_string=expr)

            if Result:
                displayHTML(f"<H2>####### DAX QUERY RESULT #######</H2>")
                display(df)

            # Wait 5 seconds for trace data to arrive
            time.sleep(5)

            # stop Trace and collect logs
            final_trace_logs:pandas.DataFrame = trace.stop()

    if Trace:
        displayHTML(f"<H2>####### SERVER TIMINGS #######</H2>")
        display(final_trace_logs)
    
    if DMV:
        displayHTML(f"<H2>####### SHOW DMV RESULTS #######</H2>")
        runDMV()

    return final_trace_logs


## Step 5: Reframe the Semantic Model

Reframe the model so it picks up the latest data from the lakehouse before running performance tests.

---

*Reframen Sie das Modell, damit es die neuesten Daten aus dem Lakehouse uebernimmt, bevor die Leistungstests durchgefuehrt werden.*

In [ ]:
# Refresh the semantic model to ensure all data changes are synchronized
labs.refresh_semantic_model(SemanticModelName)

## Step 6: Run Vertipaq Analyzer

Generate column-level storage statistics using Vertipaq Analyzer. Note that this analyses data held in the semantic model, not the Delta table directly.

---

*Erzeugen Sie spaltenbasierte Speicherstatistiken mit dem Vertipaq Analyzer. Beachten Sie, dass dies die Daten im Semantic Model analysiert, nicht direkt die Delta-Tabelle.*

In [ ]:
analyzer:dict[str,pandas.DataFrame] = labs.vertipaq_analyzer(dataset=SemanticModelName)

for key, value in analyzer.items():
    print(key)
    display(value)

In [ ]:
display(analyzer["Columns"].query("`Column Name`=='DateKey' & `Is Resident`==True"))
display(analyzer["Columns"].query("`Column Name`=='Quantity_ThisYear' & `Is Resident`==True"))

## Step 7: Run DAX Queries — Base vs. Partitioned

The following sub-steps run five DAX query patterns. Each pattern is executed first against the base (non-partitioned) table, then against the column-partitioned table, so you can compare server timing results side by side.

---

*Die folgenden Teilschritte fuehren fuenf DAX-Abfragemuster aus. Jedes Muster wird zuerst gegen die Basis-Tabelle (nicht partitioniert) und dann gegen die spaltenpartitionierte Tabelle ausgefuehrt, sodass Sie die Server-Timing-Ergebnisse direkt vergleichen koennen.*

### 7.1 Period Comparison

---

*Periodenvergleich*

#### Run Period Comparison against the base table

---

*Periodenvergleich gegen die Basis-Tabelle ausfuehren*

In [ ]:
expr:str = """

    DEFINE

        MEASURE dim_Date[Sum of Quantity] = 
            SUM(fact_myevents_1bln[Quantity_ThisYear])
            
        MEASURE dim_Date[Sum of Quantity PM] =
            CALCULATE([Sum of Quantity],PREVIOUSMONTH(dim_Date[DateKey]))

        MEASURE dim_Date[Sum of Quantity PM Delta] =
            [Sum of Quantity] - [Sum of Quantity PM]
        
        MEASURE dim_Date[Sum of Quantity PM %] =
            [Sum of Quantity PM Delta] / [Sum of Quantity]
        
    EVALUATE
        SUMMARIZECOLUMNS(
            -- GROUP BY --
            dim_Date[FirstDateofMonth] ,
            --  FILTER  --
            TREATAS({DATE(2019,1,1)} , dim_Date[FirstDateofYear] ) ,
             -- MEASURES --
            "Quantity" 				, [Sum of Quantity],
            "Quantity PM" 			, [Sum of Quantity PM],
            "Quantity PM Delta"		, [Sum of Quantity PM Delta] ,
            "Quantity PM % " 		, [Sum of Quantity PM %]
            )

"""

trace1 = runQueryWithTrace(expr,workspaceName,SemanticModelName,Result=False,DMV=False,Trace=False)
trace1 = runQueryWithTrace(expr,workspaceName,SemanticModelName,Result=False,DMV=False,Trace=False)

#### Run Period Comparison against the partitioned table

---

*Periodenvergleich gegen die partitionierte Tabelle ausfuehren*

In [ ]:
expr:str = """

    DEFINE

        MEASURE dim_Date[Sum of Quantity] = 
            SUM(fact_myevents_1bln_partitioned_datekey[Quantity_ThisYear])
            
        MEASURE dim_Date[Sum of Quantity PM] =
            CALCULATE([Sum of Quantity],PREVIOUSMONTH(dim_Date[DateKey]))

        MEASURE dim_Date[Sum of Quantity PM Delta] =
            [Sum of Quantity] - [Sum of Quantity PM]
        
        MEASURE dim_Date[Sum of Quantity PM %] =
            [Sum of Quantity PM Delta] / [Sum of Quantity]
        
    EVALUATE
        SUMMARIZECOLUMNS(
            -- GROUP BY --
            dim_Date[FirstDateofMonth] ,
            --  FILTER  --
            TREATAS({DATE(2019,1,1)} , dim_Date[FirstDateofYear] ) ,
             -- MEASURES --
            "Quantity" 				, [Sum of Quantity],
            "Quantity PM" 			, [Sum of Quantity PM],
            "Quantity PM Delta"		, [Sum of Quantity PM Delta] ,
            "Quantity PM % " 		, [Sum of Quantity PM %]
            )

"""

try:
    trace1 = runQueryWithTrace(expr,workspaceName,SemanticModelName,Result=False,DMV=False,Trace=False)
except Exception as e:
    print(e)

try:
    trace1 = runQueryWithTrace(expr,workspaceName,SemanticModelName,Result=False,DMV=False,Trace=False)
except Exception as e:
    print(e)

In [ ]:
display(trace1)
display(trace2)

### 7.2 Running Total

---

*Laufende Summe*

#### Run Running Total against the base table

---

*Laufende Summe gegen die Basis-Tabelle ausfuehren*

In [ ]:
expr:str = """

    DEFINE

        MEASURE dim_Date[Sum of Quantity] = 
            SUM(fact_myevents_1bln[Quantity_ThisYear])
            
	    MEASURE dim_Date[Sum of Quantity YTD] =
		    TOTALYTD([Sum of Quantity],dim_Date[DateKey])
	
	    MEASURE fact_myevents_1bln[Sum of Quantity QTD] =
		    TOTALQTD([Sum of Quantity],dim_Date[DateKey])	

    EVALUATE
        SUMMARIZECOLUMNS(
            -- GROUP BY --
            dim_Date[FirstDateofMonth] ,
            --  FILTER  --
            TREATAS({DATE(2019,1,1)} , dim_Date[FirstDateofYear] ) ,
             -- MEASURES --
            "Quantity" 		, [Sum of Quantity],
            "Quantity YTD" 	, [Sum of Quantity YTD] ,
            "Quantity QTD" 	, [Sum of Quantity QTD]
            )

"""
trace3 = runQueryWithTrace(expr,workspaceName,SemanticModelName,Result=False,DMV=False)

#### Run Running Total against the partitioned table

---

*Laufende Summe gegen die partitionierte Tabelle ausfuehren*

In [ ]:
expr:str="""

    DEFINE

        MEASURE dim_Date[Sum of Quantity] = 
            SUM(fact_myevents_1bln_partitioned_datekey[Quantity_ThisYear])
            
	    MEASURE dim_Date[Sum of Quantity YTD] =
		    TOTALYTD([Sum of Quantity],dim_Date[DateKey])
	
	    MEASURE fact_myevents_1bln[Sum of Quantity QTD] =
		    TOTALQTD([Sum of Quantity],dim_Date[DateKey])	

    EVALUATE
        SUMMARIZECOLUMNS(
            -- GROUP BY --
            dim_Date[FirstDateofMonth] ,
            --  FILTER  --
            TREATAS({DATE(2019,1,1)} , dim_Date[FirstDateofYear] ) ,
             -- MEASURES --
            "Quantity" 		, [Sum of Quantity],
            "Quantity YTD" 	, [Sum of Quantity YTD] ,
            "Quantity QTD" 	, [Sum of Quantity QTD]
            )

"""
trace4 = runQueryWithTrace(expr,workspaceName,SemanticModelName,Result=False,DMV=False)

In [ ]:
display(trace3)
display(trace4)

### 7.3 RANK

---

*Rangfolge*

#### Run RANK against the base table

---

*RANK gegen die Basis-Tabelle ausfuehren*

In [ ]:
expr:str = """

    DEFINE

        MEASURE dim_Date[Sum of Quantity] = 
            SUM(fact_myevents_1bln[Quantity_ThisYear])
            
        MEASURE dim_Date[Sum of Quantity Rank] =
            RANKX(ALL(dim_Geography[COUNTRY]) , [Sum of Quantity] )

    EVALUATE
        SUMMARIZECOLUMNS(
            dim_Geography[COUNTRY] ,
            TREATAS({DATE(2019,1,1)} , dim_Date[FirstDateofMonth] ) ,

            "Quantity" 		, [Sum of Quantity],
            "Rank" 			, [Sum of Quantity Rank]
            )

"""
trace5 = runQueryWithTrace(expr,workspaceName,SemanticModelName,Result=False,DMV=False)

#### Run RANK against the partitioned table

---

*RANK gegen die partitionierte Tabelle ausfuehren*

In [ ]:
expr:str = """

    DEFINE

        MEASURE dim_Date[Sum of Quantity] = 
            SUM(fact_myevents_1bln_partitioned_datekey[Quantity_ThisYear])
            
        MEASURE dim_Date[Sum of Quantity Rank] =
            RANKX(ALL(dim_Geography[COUNTRY]) , [Sum of Quantity] )

    EVALUATE
        SUMMARIZECOLUMNS(
            dim_Geography[COUNTRY] ,
            TREATAS({DATE(2019,1,1)} , dim_Date[FirstDateofMonth] ) ,

            "Quantity" 		, [Sum of Quantity],
            "Rank" 			, [Sum of Quantity Rank]
            )

"""
trace6 = runQueryWithTrace(expr,workspaceName,SemanticModelName,Result=False,DMV=False)

In [ ]:
display(trace5)
display(trace6)

### 7.4 Percent of Parent

---

*Prozent des uebergeordneten Elements*

#### Run Percent of Parent against the base table

---

*Prozent des uebergeordneten Elements gegen die Basis-Tabelle ausfuehren*

In [ ]:
expr:str = """

    DEFINE

        MEASURE dim_Date[Sum of Quantity] = 
            SUM(fact_myevents_1bln[Quantity_ThisYear])
            
	    MEASURE dim_Date[Percentage of Parent] =
		    [Sum of Quantity] / CALCULATE([Sum of Quantity],ALL(dim_Geography))

    EVALUATE
        SUMMARIZECOLUMNS(
            dim_Geography[COUNTRY] ,
            TREATAS({DATE(2019,1,1)} , dim_Date[FirstDateofMonth] ) ,
            "Quantity" 		, [Sum of Quantity],
            "% of Parent"	, [Percentage of Parent]
            )

"""
trace7 = runQueryWithTrace(expr,workspaceName,SemanticModelName,Result=False,DMV=False)

#### Run Percent of Parent against the partitioned table

---

*Prozent des uebergeordneten Elements gegen die partitionierte Tabelle ausfuehren*

In [ ]:
expr:str = """

    DEFINE

        MEASURE dim_Date[Sum of Quantity] = 
            SUM(fact_myevents_1bln_partitioned_datekey[Quantity_ThisYear])
            
	    MEASURE dim_Date[Percentage of Parent] =
		    [Sum of Quantity] / CALCULATE([Sum of Quantity],ALL(dim_Geography))

    EVALUATE
        SUMMARIZECOLUMNS(
            dim_Geography[COUNTRY] ,
            TREATAS({DATE(2019,1,1)} , dim_Date[FirstDateofMonth] ) ,
            "Quantity" 		, [Sum of Quantity],
            "% of Parent"	, [Percentage of Parent]
            )

"""
trace8 = runQueryWithTrace(expr,workspaceName,SemanticModelName,Result=False,DMV=False)

In [ ]:
display(trace7)
display(trace8)

### 7.5 All Measures Combined

Run all five query patterns in a single DAX query to compare total execution time.

---

*Fuehren Sie alle fuenf Abfragemuster in einer einzigen DAX-Abfrage aus, um die Gesamtausfuehrungszeit zu vergleichen.*

#### Run all measures against the base table

---

*Alle Measures gegen die Basis-Tabelle ausfuehren*

In [ ]:
expr:str = """

    DEFINE

        MEASURE dim_Date[Sum of Quantity] = 
            SUM(fact_myevents_1bln[Quantity_ThisYear])
            
        MEASURE dim_Date[Percentage of Parent] =
            [Sum of Quantity] / CALCULATE([Sum of Quantity],ALL(dim_Geography))

        MEASURE dim_Date[Sum of Quantity Rank] =
            RANKX(ALL(dim_Geography[COUNTRY]) , [Sum of Quantity] )

        MEASURE dim_Date[Sum of Quantity YTD] =
            TOTALYTD([Sum of Quantity],dim_Date[DateKey])
        
        MEASURE dim_Date[Sum of Quantity QTD] =
            TOTALQTD([Sum of Quantity],dim_Date[DateKey])	

        MEASURE dim_Date[Sum of Quantity PM] =
            CALCULATE([Sum of Quantity],PREVIOUSMONTH(dim_Date[DateKey]))

        MEASURE dim_Date[Sum of Quantity PM Delta] =
            [Sum of Quantity] - [Sum of Quantity PM]
        
        MEASURE dim_Date[Sum of Quantity PM %] =
            [Sum of Quantity PM Delta] / [Sum of Quantity]

    EVALUATE
        SUMMARIZECOLUMNS(
            dim_Geography[COUNTRY] ,
            TREATAS({DATE(2019,1,1)} , dim_Date[FirstDateofMonth] ) ,
            "Quantity" 				, [Sum of Quantity],
            "% of Parent"			, [Percentage of Parent],
            "Rank" 					, [Sum of Quantity Rank],
            "Quantity YTD" 			, [Sum of Quantity YTD] ,
            "Quantity QTD" 			, [Sum of Quantity QTD]	,	
            "Quantity PM" 			, [Sum of Quantity PM],
            "Quantity PM Delta"		, [Sum of Quantity PM Delta] ,
            "Quantity PM %" 		, [Sum of Quantity PM %]
            )

"""
trace9 = runQueryWithTrace(expr,workspaceName,SemanticModelName,Result=False,DMV=False)

#### Run all measures against the partitioned table

---

*Alle Measures gegen die partitionierte Tabelle ausfuehren*

In [ ]:
expr:str = """

    DEFINE

        MEASURE dim_Date[Sum of Quantity] = 
            SUM(fact_myevents_1bln_partitioned_datekey[Quantity_ThisYear])
            
        MEASURE dim_Date[Percentage of Parent] =
            [Sum of Quantity] / CALCULATE([Sum of Quantity],ALL(dim_Geography))

        MEASURE dim_Date[Sum of Quantity Rank] =
            RANKX(ALL(dim_Geography[COUNTRY]) , [Sum of Quantity] )

        MEASURE dim_Date[Sum of Quantity YTD] =
            TOTALYTD([Sum of Quantity],dim_Date[DateKey])
        
        MEASURE dim_Date[Sum of Quantity QTD] =
            TOTALQTD([Sum of Quantity],dim_Date[DateKey])	

        MEASURE dim_Date[Sum of Quantity PM] =
            CALCULATE([Sum of Quantity],PREVIOUSMONTH(dim_Date[DateKey]))

        MEASURE dim_Date[Sum of Quantity PM Delta] =
            [Sum of Quantity] - [Sum of Quantity PM]
        
        MEASURE dim_Date[Sum of Quantity PM %] =
            [Sum of Quantity PM Delta] / [Sum of Quantity]

    EVALUATE
        SUMMARIZECOLUMNS(
            dim_Geography[COUNTRY] ,
            TREATAS({DATE(2019,1,1)} , dim_Date[FirstDateofMonth] ) ,
            "Quantity" 				, [Sum of Quantity],
            "% of Parent"			, [Percentage of Parent],
            "Rank" 					, [Sum of Quantity Rank],
            "Quantity YTD" 			, [Sum of Quantity YTD] ,
            "Quantity QTD" 			, [Sum of Quantity QTD]	,	
            "Quantity PM" 			, [Sum of Quantity PM],
            "Quantity PM Delta"		, [Sum of Quantity PM Delta] ,
            "Quantity PM %" 		, [Sum of Quantity PM %]
            )

"""
trace10 = runQueryWithTrace(expr,workspaceName,SemanticModelName,Result=False,DMV=False)

In [ ]:
display(trace9)
display(trace10)

## Step 8: Stop the Spark Session

---

*Spark-Sitzung beenden.*

In [ ]:
mssparkutils.session.stop()

## Lab Summary

In this lab you measured the performance difference between a standard table and a column-partitioned table using five common DAX query patterns: period comparison, running total, RANK, percent of parent, and a combined query. Server timing traces were captured for each execution to quantify the impact.

**Key Takeaways**
- Column partitioning reorganises how data is stored on disk, which can significantly reduce the amount of data scanned during query execution.
- Cold-cache performance (first query after a reframe) is where partitioning tends to show the largest benefit, because less data needs to be paged into memory.
- Warm-cache queries benefit less from partitioning since data is already resident in memory, but I/O savings can still be observed.
- Vertipaq Analyzer provides the column-level storage statistics needed to identify partitioning candidates.
- Server timing traces are essential for objective performance comparison — avoid relying on perceived speed alone.

**Next Lab:** Lab 7 explores splitting high cardinality columns to reduce model size and improve query performance.

---

*In diesem Lab haben Sie den Leistungsunterschied zwischen einer Standard-Tabelle und einer spaltenpartitionierten Tabelle anhand von fuenf gaengigen DAX-Abfragemustern gemessen: Periodenvergleich, laufende Summe, RANK, Prozent des uebergeordneten Elements und eine kombinierte Abfrage. Fuer jede Ausfuehrung wurden Server-Timing-Traces erfasst, um die Auswirkungen zu quantifizieren.*

**Wichtige Erkenntnisse**
- Spaltenpartitionierung organisiert die Datenspeicherung auf der Festplatte neu, was die Menge der bei der Abfrageausfuehrung gescannten Daten erheblich reduzieren kann.
- Die Cold-Cache-Leistung (erste Abfrage nach einem Reframe) profitiert am meisten von der Partitionierung, da weniger Daten in den Arbeitsspeicher geladen werden muessen.
- Warm-Cache-Abfragen profitieren weniger, da die Daten bereits im Speicher liegen, aber E/A-Einsparungen koennen dennoch beobachtet werden.
- Der Vertipaq Analyzer liefert die spaltenbasierten Speicherstatistiken, die zur Identifizierung von Partitionierungskandidaten erforderlich sind.
- Server-Timing-Traces sind fuer einen objektiven Leistungsvergleich unerlaesslich — verlassen Sie sich nicht nur auf die gefuehlte Geschwindigkeit.

**Naechstes Lab:** Lab 7 untersucht das Aufteilen von Spalten mit hoher Kardinalitaet, um die Modellgroesse zu reduzieren und die Abfrageleistung zu verbessern.